# Generating Counterfactuals

In this notebook, we will focus on generating counterfactuals from individual
datapoints. This will be implemented for the following models:

- Naive Bayes
- Fair Bayesian Network
- Fair Random Forest Classifier

By generating counterfactuals, we hope to gain insight into how the model uses
the different attributes in it's decisions.

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.bayesnet import latentLabelClassifier, interpretableNaiveBayes
import pandas as pd
from random import sample
import numpy as np
from forseti.datproc import translate_categorical

df = pd.read_csv("data/adult.csv")

clf = interpretableNaiveBayes()

tmp = df.copy(deep=True)
label = "income"
clf.train(label, df, "NB")
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)

## Naive Bayes, Black Female Low Income

In [2]:
idx = df[
    (df.gender == 'Female') & # Female
    (df.race == 'Black') & # Black
    (df.income == '<=50K') # Low Income
].sample(1).index[0]

datapoint = pd.DataFrame(tmp.iloc[idx]).T
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,


## Show the Datapoint

In [3]:
datapoint

,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,hours-per-week
6656,"(16.927, 31.6]",Private,11th,Never-married,Adm-clerical,Own-child,Black,Female,"(-4460.355, 16515.0]","(0.902, 20.6]"


## Show Counterfactuals

In [4]:
R[
    (R['O1'] <= 0.5)
].sort_values(['O1', 'O3'])

,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,hours-per-week,O1,O2,O3,O4
102,"(16.927, 31.6]",Private,11th,Never-married,Adm-clerical,Own-child,Black,Male,"(58257.0, 79128.0]","(0.902, 20.6]",0.0,0.8,2,0.0
126,"(16.927, 31.6]",Private,11th,Never-married,Adm-clerical,Own-child,Black,Male,"(79128.0, 99999.0]","(0.902, 20.6]",0.0,0.8,2,0.0
123,"(16.927, 31.6]",Private,11th,Never-married,Priv-house-serv,Own-child,Black,Male,"(79128.0, 99999.0]","(0.902, 20.6]",0.0,0.7,3,0.0
124,"(16.927, 31.6]",Self-emp-inc,11th,Never-married,Adm-clerical,Own-child,Black,Male,"(58257.0, 79128.0]","(0.902, 20.6]",0.0,0.7,3,0.0
173,"(16.927, 31.6]",Private,11th,Never-married,Priv-house-serv,Own-child,Black,Male,"(79128.0, 99999.0]","(0.902, 20.6]",0.0,0.7,3,0.0
139,"(16.927, 31.6]",Self-emp-inc,11th,Never-married,Priv-house-serv,Own-child,Black,Male,"(79128.0, 99999.0]","(0.902, 20.6]",0.0,0.6,4,0.0


## Naive Bayes Without Sensitive, Black Female Low Income

In [5]:
df = pd.read_csv("data/adult.csv")

clf = interpretableNaiveBayes()

tmp = df.copy(deep=True)
label = "income"
df[['race', 'gender']] = np.random.permutation(df[['race', 'gender']])
clf.train(label, df, "NB")
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)

datapoint = pd.DataFrame(tmp.iloc[idx]).T
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,
/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:472: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi.values.sum()


## Show the datapoint

In [ ]:
datapoint

## Show the counterfactuals

In [ ]:
R[
    (R['O1'] <= 0.5)
].sort_values(['O1', 'O3'])

## Fair Bayesian Network

In [ ]:
df = pd.read_csv("data/adult.csv")
tmp = df.copy(deep=True)
tmp, _ = translate_categorical(tmp)
tmp = tmp.drop(label, axis=1)
label = "income"
sensitives = ['gender', 'race']

clf = latentLabelClassifier(
    df, 
    sensitives, 
    label
)

datapoint = pd.DataFrame(tmp.iloc[idx]).T
clf.load('trained-models/fair_model.sav')
datapoint, R = clf.generateCounterfactuals(datapoint, candidates=100, gen=5)

## Show the datapoint

In [ ]:
datapoint

## Show the counterfactuals

In [ ]:
R.sort_values(['O1', 'O3'])